# CIA World Factbook project

In this project, we use SQL to explore and analyze data from the CIA World Factbook: https://www.cia.gov/library/publications/the-world-factbook/

The demographic information contained in this database:
 - population (as of 2015)
 - population_growth (the annual population growth rate as percentage)
 - area (the total land and water area)

Connect Jupyter Notebook to the database file

In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Query our database to find out the name of the table and how it looks like

In [5]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Return the first 5 rows of the facts table 

In [6]:
%%sql
SELECT *
  FROM facts
 LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Calculate some summary statistics and look for any outlier countries:
- minimum population
- maximum population
- minimum population growth
- maximum population growth

In [13]:
%%sql
SELECT MIN(population) Min_population,
       MAX(population) Max_population,
       MIN(population_growth) Min_population_growth,
       MAX(population_growth) Max_population_growth 
  FROM facts;

Done.


Min_population,Max_population,Min_population_growth,Max_population_growth
0,7256490011,0.0,4.02


Some comments after looking at this summary:
- There is a country with a population of 0
- There is a country with a population of over 7 billion people

We will take a look at these countries without using the specific values.

In [14]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [15]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The country with 0 population is Antarctica and there is a row for the whole world that containes the 7.2 billion population. 

Both of these make sense now, but let's recalculate the summary statistics from before, excluding the row for the whole world.

In [16]:
%%sql
SELECT MIN(population) Min_population,
       MAX(population) Max_population,
       MIN(population_growth) Min_population_growth,
       MAX(population_growth) Max_population_growth 
  FROM facts
 WHERE population<7256490011

Done.


Min_population,Max_population,Min_population_growth,Max_population_growth
0,1367485388,0.0,4.02


Calculate the average value for population and area, rounded to 2 decimals

In [19]:
%%sql
SELECT ROUND(AVG(population), 2) Average_population, 
       ROUND(AVG(area), 2) Average_area
  FROM facts;

Done.


Average_population,Average_area
62094928.32,555093.55


Find the countries that are densely populated:
- Countries that have above values for population and
- Below average values for area

In [22]:
%%sql
SELECT name, area, population
  FROM facts
 WHERE population > (SELECT AVG(population) FROM facts) AND
       area < (SELECT AVG(area) FROM facts)
 ORDER BY population

Done.


name,area,population
United Kingdom,243610,64088222
Thailand,513120,67976405
Germany,357022,80854408
Vietnam,331210,94348835
Philippines,300000,100998376
Japan,377915,126919659
Bangladesh,148460,168957745


 What country has the most people?
 What country has the highest growth rate?

%%sql
SELECT name AS Most_populated_country
  FROM facts
 WHERE population == (SELECT MAX(population) FROM facts WHERE name<>'World') 
       

In [29]:
%%sql
SELECT name, population_growth AS Highest_pop_growth
  FROM facts
 WHERE population_growth == (SELECT MAX(population_growth) FROM facts) 

Done.


name,Highest_pop_growth
South Sudan,4.02


- Which countries have the highest ratios of water to land? 
- Which countries have more water than land?

In [37]:
%%sql
SELECT name, area_water, area_land
FROM facts


Done.


name,area_water,area_land
Afghanistan,0,652230
Albania,1350,27398
Algeria,0,2381741
Andorra,0,468
Angola,0,1246700
Antigua and Barbuda,0,442
Argentina,43710,2736690
Armenia,1540,28203
Australia,58920,7682300
Austria,1426,82445


Some values are set to none instead of 0 and we need to change that.

In [42]:
%%sql
UPDATE facts SET area_water=0 WHERE area_water IS NULL


0 rows affected.


[]

In [41]:
%%sql
UPDATE facts SET area_land=0 WHERE area_land IS NULL

15 rows affected.


[]

In [66]:
%%sql
SELECT name, area_water, area_land,
       CAST (area_water AS FLOAT)/CAST (area_land AS FLOAT) Ratio
  FROM facts
 ORDER BY Ratio DESC
 LIMIT 10
 

Done.


name,area_water,area_land,Ratio
British Indian Ocean Territory,54340,60,905.6666666666666
Virgin Islands,1564,346,4.520231213872832
Puerto Rico,4921,8870,0.5547914317925592
"Bahamas, The",3870,10010,0.3866133866133866
Guinea-Bissau,8005,28120,0.2846728307254623
Malawi,24404,94080,0.25939625850340137
Netherlands,7650,33893,0.22571032366565366
Uganda,43938,197100,0.22292237442922375
Eritrea,16600,101000,0.16435643564356436
Liberia,15049,96320,0.15623961794019933


These are the top 10 countries with the highest ratio of water to land: British Indian Ocean Territory, Virgin Islands, Puerto Rico, The Bahamas, Guinea-Bissau, Malawi, Netherlands, Uganda, Eritrea, Liberia

In [67]:
%%sql
SELECT name, area_water, area_land
  FROM facts
 WHERE area_water>area_land

Done.


name,area_water,area_land
Ethiopia,104300,0
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


It looks like here are some wrong data in our dataset..Ethiopia should have land area. Disconsidering that, we only have 2 countries with more water than land: British Indian Ocean Territory and Virgin Islands.

Which countries will add the most people to their population next year?

In [78]:
%%sql
SELECT name, population, population_growth,
       population*population_growth/100 people_added
  FROM facts
  WHERE name<>'World'
 ORDER BY people_added DESC
 LIMIT 6

Done.


name,population,population_growth,people_added
India,1251695584,1.22,15270686.1248
China,1367485388,0.45,6153684.246
Nigeria,181562056,2.45,4448270.372
Pakistan,199085847,1.46,2906653.3662
Ethiopia,99465819,2.89,2874562.1691
Bangladesh,168957745,1.6,2703323.92


Top 5 countries that will add the most people to their population next year are: India, China, Nigeria, Pakistan, Ethiopia

Which countries have a higher death rate than birth rate?

In [75]:
%%sql
SELECT name, birth_rate, death_rate
  FROM facts
 WHERE birth_rate<death_rate

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


In [77]:
%%sql
SELECT COUNT(*)
  FROM facts

Done.


COUNT(*)
261


There are 23 countries out of 261 with higher death rate and birth rate.

What countries have the highest population/area ratio?

In [82]:
%%sql
SELECT name, population, area, population/area ratio
  FROM facts
 ORDER BY ratio DESC
 LIMIT 10
 

Done.


name,population,area,ratio
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


The most populated countries/area in our 2015 dataset where: Macau, Monaco, Singapore, Hong Kong, Gaza Strip.

# Conclusion

This was a practice and play project to understand the basic SQL syntax and see how basic summary statistics can be done using SQL.

I will definitely explore more of it, but for a first run it seems very straight forward and easy to use.